In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

# Iniciando os dados para serem analizados

In [2]:
df = pd.read_csv("soc-sign-bitcoinotc.csv", encoding="utf-8")

Removendo dados com nota menor que 8

In [3]:
df = df[df.rating >= 8]

Filtrando apenas as colunas relevantes ("source" e "terget")

In [5]:
df = df.loc[:, 'source':'target']

Listas de todos os inventidores e quantidade de investidores

In [113]:
investors = list(set(df.source) | set(df.target))
num_investors = len(investors)

Dicionario que ralaciona investidor com quantidade de avaliações realizadas

In [114]:
num_rating = df.groupby(['source'])['target'].apply(list).to_dict()

Montando as matrizes "a", "b" e "m" e o vetor normalizado "v"

In [115]:
matrix_a = np.zeros((num_investors, num_investors))

for i in range(num_investors) :
    source = investors[i]
    for j in range(num_investors) :
        target = investors[j]
        if(source in num_rating and target in num_rating[source]) :
            matrix_a[i][j] = 1.0 / float(len(num_rating[source]))

In [116]:
matrix_b = (1.0/float(num_investors))*np.matrix(np.ones((num_investors, num_investors)))

In [117]:
d = 0.15
matrix_m = ((1-d)*matrix_a) + (d*matrix_b)

In [118]:
vector_v = (1.0/float(num_investors))*np.matrix(np.ones((num_investors, 1)))

# Função para calcular o Pagerank

In [119]:
count = 0
def pagerank(v, m) :
    global count
    count += 1
    if (sum(abs((m*v)-v)) > 0.001) :
        return pagerank(m*v, m)
    else :
        return m*v

# Executando o algoritimo do Pagerank
Execução do pagrank + exportação dos resultados

In [120]:
rank = pagerank(vector_v, matrix_m)
rank = [matr.item(0,0) for matr in rank]
ranking = {
    'investor': investors,
    'pagerank': rank
}
pd.DataFrame(ranking).to_csv('pagerank.csv', index=False)

# Feitos os passos acima, responda as seguintes perguntas:

### Quantas iterações o PageRank precisou rodar até atingir convergência?

In [121]:
print(count , "iterações.")

35 iterações.


### Quais os 5 investidores mais importantes segundo o PageRank? Quais seus valores de PageRank?

In [123]:
pd.read_csv('pagerank.csv').sort_values(by=['pagerank']).head(5)

,investor,pagerank
161,4393,0.000001
429,2896,0.000001
654,5521,0.000001
185,2419,0.000001
808,3781,0.000001


### Como você poderia usar o PageRank caso você fosse um investidor em bitcoins?

Os investidores com maior pagerank são os mais confiáveis para realizar negociações.